# Cloud Applications for IoT

![Cloud to Ground](https://lecmaterials.s3-us-west-1.amazonaws.com/Cloud2Ground.png)

# What is the Cloud to an IoT Device

* Persistence
* Intelligence

# Cloud Connection - Protocols Overview

* HTTP
* MQTT
* WebSockets

# HTTP

* Method
* Host
* Resource
* Headers
* Query Parameters

# An Aside - IoT Cloud Platforms

* AWS
* Azure
* GCP
* ThingWorx

# Cloud Interface

* Machine Interfaces
  * Strictly defined (APIs, JSON, HDF5)  
* Human Interfaces
  * Web or Native App

# Example Cloud Application

* Internet accessible cloud API
* Data storage and retrieval

# API Endpoint with Compute

* Serverless Framework
    * API Gateway
    * Lambda - Python

* https://tutorialedge.net/cloud/aws/python-based-lambda-tutorial/

# serverless.yml
```
service: hello-world
provider:
  name: aws
  runtime: python3.6
  region: us-west-1
```

# serverless.yml cont. - Function Definition

```
functions:
  hello:
    handler: simple.hello
    events:
      - http:
          path: hello
          method: get
```

# simple.py - AWS Lambda Handler

In [ ]:
def hello(event, context):

    response = {
        "statusCode": 200,
        "body": 'hello, world'
    }

    return response

# Cloud Storage

* S3
* DynamoDB
* Relational Database Service (RDS)
* ElasticSearch, Aurora Serverless

# S3 Upload  Example

In [15]:
import boto3
import json
from io import BytesIO
bucket = boto3.resource("s3").Bucket("lecmaterials")
key = "graphthis.json"
obj = BytesIO()
obj.write(json.dumps("GIANTMETEOR_2020").encode())   
obj.seek(0)
print('Saving file s3://%s/%s' % (bucket, key))
bucket.Object(key).upload_fileobj(obj)
print("Upload complete")

Saving file s3://s3.Bucket(name='lecmaterials')/graphthis.json
Upload complete


# S3 Download Example

In [16]:
obj = BytesIO()
bucket.Object(key).download_fileobj(obj)
obj.seek(0)     # move the cursor to the beginning of the file object
stored = json.loads(obj.read().decode('utf-8'))   # return as a string
print("Recovered: {}".format(stored))

Recovered: GIANTMETEOR_2020


# Expanded Example API

* POST Data: The API will provide an endpoint to accept a string of data and store it at the provided path.
* GET Data: The API will provide an endpoint to accept a path and return the string of data stored there.

# Events will come in with the format:
```
{
    "resource"
    "pathParameters"
    "httpMethod"
    "headers"
    "queryStringParameters"
    "body"
}
```

# Expanded AWS API Lambda Handler

In [ ]:
def handle_request(event, context):
    print('Received Event: %s' % json.dumps(event))
    if event['httpMethod'].lower() == 'get':
        return handle_get(event)
    elif event['httpMethod'].lower() == 'post':
        return handle_post(event)
    else:
        return {'statusCode': 415, 'body': 'method not allowed'}

# Handle POST

In [ ]:
def handle_post(event):
    key = event['pathParameters']['key']
    # contents needs to be in bytes, not str
    contents = event['body'].encode('utf-8') if not isinstance(event['body'], bytes) else event['body']
    try:
        return {'statusCode': 200, 'body': put_s3_file_contents(key, contents)}
    except Exception as e:
        return {'statusCode': 500, 'body': 'Something went wrong:\n\t%s' % str(e)}

# Handle S3 Storage

In [ ]:
def put_s3_file_contents(key, contents):
    obj = BytesIO()
    obj.write(contents)
    obj.seek(0)
    bucket = os.environ['S3_BUCKET']
    print('Saving file s3://%s/%s' % (bucket, key))
    boto3.resource('s3').Bucket(bucket).Object(key).upload_fileobj(obj)

# Handle GET

In [ ]:
def handle_get(event):
    key = event['pathParameters']['key']
    try:
        return {'statusCode': 200, 'body': get_s3_file_contents(key)}
    except ClientError as e:
        if '404' in str(e):
            return {'statusCode': 404, 'body': 'File with name %s not found' % key}
        else:
            return {'statusCode': 500, 'body': 'Something went wrong:\n\t%s' % str(e)}
    except Exception as e2:
        return {'statusCode': 500, 'body': 'Something went wrong:\n\t%s' % str(e2)}

# Handle S3 Retrieval

In [ ]:
def get_s3_file_contents(key):
    obj = BytesIO()
    bucket = os.environ['S3_BUCKET']
    print('Downloading file s3://%s/%s' % (bucket, key))
    boto3.resource('s3').Bucket(bucket).Object(key).download_fileobj(obj)
    obj.seek(0)     # move the cursor to the beginning of the file object
    return obj.read().decode('utf-8')   # return as a string

# Human Interface - Web Application

* Hosting a Web App
    * Apache, Django, LambdaPage
* Building a Web App
    * HTML, CSS, Javascript

# Further Reading

* Cloud Data Storage - DynamoDB, ElasticSearch, RDS
* Cloud Compute - Lambda (Lambda Layers), EC2
* Python Libraries - Boto3, LambdaPage, Vaex, BeautifulSoup